In [ ]:
!git clone https://github.com/mueedurrehman/distiller.git
%cd distiller
!pip3 install -e .

Cloning into 'distiller'...
remote: Enumerating objects: 6174, done.
remote: Total 6174 (delta 0), reused 0 (delta 0), pack-reused 6174
Receiving objects: 100% (6174/6174), 39.82 MiB | 34.85 MiB/s, done.
Resolving deltas: 100% (4331/4331), done.
/content/distiller
Obtaining file:///content/distiller
     |████████████████████████████████| 2.1MB 18.6MB/s 
     |████████████████████████████████| 734.6MB 25kB/s 
     |████████████████████████████████| 10.2MB 41.3MB/s 
     |████████████████████████████████| 460kB 56.8MB/s 
     |████████████████████████████████| 110.5MB 37kB/s 
     |████████████████████████████████| 51kB 9.3MB/s 
     |████████████████████████████████| 798kB 53.5MB/s 
     |████████████████████████████████| 112kB 55.0MB/s 
     |████████████████████████████████| 1.7MB 55.1MB/s 
     |████████████████████████████████| 235kB 53.5MB/s 
     |████████████████████████████████| 153kB 57.4MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time
import math
import yaml
import distiller
from collections import OrderedDict
import os
import pandas as pd
import distiller.models
import distiller.quantization
from collections import OrderedDict
import copy

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Pruning Schedules/TuningExperiments/2020.12.05-221020/best.pth.tar" .
# !cp ../../../drive/MyDrive/CS6787\ -\ Final\ Project/Google\ Colab\ Scripts/Quantization\ Models/Resnet56\ Quantization\ Stats/resnet56_quant_stats.yaml .

TuningExperiments/2020.12.05-221020

92.250 89.25

In [ ]:
!python3 /content/distiller/examples/classifier_compression/compress_classifier.py -a resnet56_cifar --compress="/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/QA/Asymmetric/4bitAsymmetricUQA.yaml" ./data --resume="best.pth.tar" -o "/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/SparseQAAsymmetric" --qe-stats-file="/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/acts_quantization_stats.yaml" --qe-clip-acts=avg --epochs 25 --vs=0 --lr=0.0001 --prioritizeAccuracy

Streaming output truncated to the last 5000 lines.
| 170 | module.layer3.1.conv2.weight_scale             | (64, 1, 1, 1)  |            64 |             64 |    0.00000 |    0.00000 |  0.00000 |  0.00000 |  0.00000 |    0.00000 |  11.44295 |  46.24758 |   46.24758 |
| 171 | module.layer3.1.conv2.weight_zero_point        | (64, 1, 1, 1)  |            64 |             62 |    0.00000 |    0.00000 |  0.00000 |  3.12500 |  3.12500 |    3.12500 |   1.59861 |  -7.12500 |    7.12500 |
| 172 | module.layer3.2.conv1.float_weight             | (64, 64, 3, 3) |         36864 |           3516 |    0.00000 |    0.00000 |  6.25000 | 54.34570 |  0.00000 |   90.46224 |   0.03026 |  -0.00049 |    0.00871 |
| 173 | module.layer3.2.conv1.weight                   | (64, 64, 3, 3) |         36864 |           3489 |    0.00000 |    0.00000 |  6.25000 | 54.54102 |  0.00000 |   90.53548 |   0.03034 |  -0.00051 |    0.00871 |
| 174 | module.layer3.2.conv1.weight_scale             | (64, 1, 1, 1)  |            

In [ ]:
!python3 /content/distiller/examples/classifier_compression/compress_classifier.py -a resnet56_cifar --compress="/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/QA/Asymmetric/4bitAsymmetricUQA.yaml" ./data --resume="best.pth.tar" -o "/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/SparseQAAsymmetric" --qe-stats-file="/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/acts_quantization_stats.yaml" --qe-clip-acts=none --epochs 25 --vs=0 --lr=0.0001 --prioritizeAccuracy

Streaming output truncated to the last 5000 lines.
| 170 | module.layer3.1.conv2.weight_scale             | (64, 1, 1, 1)  |            64 |             64 |    0.00000 |    0.00000 |  0.00000 |  0.00000 |  0.00000 |    0.00000 |  11.43379 |  46.24645 |   46.24645 |
| 171 | module.layer3.1.conv2.weight_zero_point        | (64, 1, 1, 1)  |            64 |             62 |    0.00000 |    0.00000 |  0.00000 |  3.12500 |  3.12500 |    3.12500 |   1.59861 |  -7.12500 |    7.12500 |
| 172 | module.layer3.2.conv1.float_weight             | (64, 64, 3, 3) |         36864 |           3516 |    0.00000 |    0.00000 |  6.25000 | 54.34570 |  0.00000 |   90.46224 |   0.03026 |  -0.00049 |    0.00871 |
| 173 | module.layer3.2.conv1.weight                   | (64, 64, 3, 3) |         36864 |           3490 |    0.00000 |    0.00000 |  6.25000 | 54.54102 |  0.00000 |   90.53277 |   0.03033 |  -0.00051 |    0.00871 |
| 174 | module.layer3.2.conv1.weight_scale             | (64, 1, 1, 1)  |            